In [ ]:
import pandas as pd
import os
import gzip
import pickle
import openai
import re
import copy
from tqdm import tqdm
from torch import save,load

concatenated_threads = pd.read_csv('../2_train_set_creation/concatenated_threads.csv')

id = 0

hardware_instances = []
hardware_instances_inc_threads = []

for i in range(1, 14):
    texts = load(f'output_chunks/outputs_chunk_{i}.pt')
    for text in texts:
        output = text.split("Hardware names:")[1]
        try:
            date = text.split("Date:")[1].split("Topic:")[0].strip()
        except:
            print("something went wrong")
            # print(text)
            id+=1
            continue

        # print("id: ", id)
        # print(date)
        # print(concatenated_threads.iloc[id]['date'])
        # print(output)

        date = concatenated_threads.iloc[id]['date']

        sp = output.split("Hardware ownership:")
        if len(sp) != 2:
            print("something went wrong2")
            # print(text)
            id+=1
            continue
        names = sp[0].strip()
        ownerships = sp[1].strip()
        for (name, ownership) in zip(names.split(","), ownerships.split(",")):
            n = name.strip().lower()
            o = ownership.strip()
            if o == "True":
                hardware_instances.append((date, n))

        for (name, ownership) in zip(names.split(","), ownerships.split(",")):
            n = name.strip().lower()
            o = ownership.strip()
            if o == "True":
                hardware_instances_inc_threads.append((date, text))
                break
        
        id+=1
        # break

    # break
        
df = pd.DataFrame(hardware_instances, columns=['date', 'hardware'])
df = df.sort_values(by=['date'])
hardware_instances

In [ ]:
df.to_csv('hardware_instances.csv', index=False)

In [ ]:
df = pd.DataFrame(hardware_instances_inc_threads, columns=['date', 'thread'])
df = df.sort_values(by=['date'])
hardware_instances_inc_threads

In [ ]:
df.to_csv('hardware_instances_inc_threads.csv', index=False)